In [86]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar
from dateutil.relativedelta import relativedelta

df = pd.read_excel(r'C:\Users\ssing296\Desktop\Python-Coursera\97-98 COVID_QA_Final Output_Testing details_1.xlsx')

us_positivity = pd.read_excel(r'C:\Users\ssing296\Desktop\Python-Coursera\dev_(CTP Replacement) COVID New Cases, Deaths, Tests w_ Moving Averages w_ US Total.xlsx')


ignore_state_list = ['SC']

num2words = {1: 'One', 2: 'Two', 3: 'Three', 4: 'Four', 5: 'Five', \
             6: 'Six', 7: 'Seven', 8: 'Eight', 9: 'Nine', 10: 'Ten', \
            11: 'Eleven', 12: 'Twelve', 13: 'Thirteen', 14: 'Fourteen', \
            15: 'Fifteen', 16: 'Sixteen', 17: 'Seventeen', 18: 'Eighteen', \
            19: 'Nineteen', 20: 'Twenty', 30: 'Thirty', 40: 'Forty', \
            50: 'Fifty', 60: 'Sixty', 70: 'Seventy', 80: 'Eighty', \
            90: 'Ninety', 0: 'Zero'}

#n2w_dict = {0: 'none',1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', \
#            6: 'six', 7: 'seven', 8: 'eight', 9: 'nine'}



def n2w(n):
  try:
    return (num2words[n])
  except KeyError:
    try:
      return num2words[n-n%10] + num2words[n%10].lower()
    except KeyError:
      return 'Number out of range'

def us_states_positivity_greater(df,threshold,ignore_state_list,ignore_states_flag):
    
    if ignore_states_flag == 1:
      df = df[~df['State'].isin(ignore_state_list)]
    else:
      print('No states removed!')
    
    filter_cond = (df['Latest Date Rank'] >= 1) & (df['Latest Date Rank'] <= 7)
    
    filtered_data = df[filter_cond]
    
    state_list = filtered_data.groupby(['State'])['State'].max().tolist()
    positive_increase_state = filtered_data.groupby(['State'])['positiveIncrease'].sum().tolist()
    total_test_increase_state = filtered_data.groupby(['State'])['totalTestResultsIncrease'].sum().tolist()
    
    positivity_rate = []
    
    for i in range(0,len(state_list)):
        positivity_rate.append(positive_increase_state[i]/total_test_increase_state[i]*100)
        
    pos_greater_pc_dict = {}
    pos_lessor_pc_dict ={}
    
    for i in range(0,len(state_list)):
        if(positivity_rate[i] >= threshold):
            pos_greater_pc_dict[state_list[i]] = positivity_rate[i]
        else:
            pos_lessor_pc_dict[state_list[i]] = positivity_rate[i]
        
 
    pos_greater_pc_dict = {k: v for k, v in sorted(pos_greater_pc_dict.items(),reverse = True, key=lambda x: x[1])}
    pos_lessor_pc_dict = {k: v for k, v in sorted(pos_lessor_pc_dict.items(),reverse = True, key=lambda x: x[1])[:5]}
    
    print(len(pos_lessor_pc_dict))
    
    expression_greater = []
    expression_lessor = []
    for i in range(0, len(pos_greater_pc_dict),1):
        expression_greater.append(list(pos_greater_pc_dict)[i])
        expression_greater.append((list(pos_greater_pc_dict.values())[i]))
        

    for i in range(0,len(pos_lessor_pc_dict),1):
        expression_lessor.append(list(pos_lessor_pc_dict)[i])
        expression_lessor.append((list(pos_lessor_pc_dict.values())[i]))
        
    print(expression_lessor)
    
    if len(pos_greater_pc_dict) == 0:
        comment_length = len(pos_lessor_pc_dict) - 1
        comment  = 'States with the highest 7-day positivity rate:' +comment_length*(', {} ({:.1f}%)') + ' and {} ({:.1f}%)'+ str("") + '.'.format(*expression_lessor)
        a = comment.format(*expression_lessor)
        a = a.replace(':,',':')
    
    elif len(pos_greater_pc_dict) == 1:
        a='Only {} ({:.1f}%) had a 7-day average positivity rate greater than 20%.'.format(*expression_greater)
       
    elif len(pos_greater_pc_dict) == 2:
        comment=n2w(len(pos_greater_pc_dict))+" States had a 7-day average positivity rate greater than 20%: " + '{} ({:.1f}%) and {} ({:.1f}%)' + str("") + '.'.format(*expression_greater)

        a = comment.format(*expression_greater)
    
    elif len(pos_greater_pc_dict) >2:
        comment_length = len(pos_greater_pc_dict) - 2

        comment=n2w(len(pos_greater_pc_dict))+" States had a 7-day average positivity rate greater than 20%: " + '{} ({:.1f}%)' + comment_length*(', {} ({:.1f}%)') + ' and {} ({:.1f}%)' + str("") + '.'.format(*expression_greater)
        
        a = comment.format(*expression_greater)


    return a
            
    


States_Comments_Positivity = us_states_positivity_greater(us_positivity, 20,ignore_state_list,0)

print(States_Comments_Positivity)

No states removed!
5
['SC', 10.579183904282866, 'SD', 9.179852639207633, 'NJ', 8.203368683718029, 'NE', 7.039070067807556, 'MI', 6.962560812240006]
States with the highest 7-day positivity rate: SC (10.6%), SD (9.2%), NJ (8.2%), NE (7.0%) and MI (7.0%).
